# How to use `seesus`

More description coming up

In [1]:
from seesus import SeeSus

In [16]:
text = "our goal is to end hunger globally"

In [17]:
result = SeeSus(text)

In [18]:
print(result.sdg)
print(result.sdg_desc)

['SDG2']
['Zero Hunger']


In [19]:
print(result.target)
print(result.target_desc)

['SDG2_1']
['By 2030, end hunger and ensure access by all people, in particular the poor and people in vulnerable situations, including infants, to safe, nutritious and sufficient food all year round']


In [21]:
print(result.see)

{'social_sustainability': True, 'environmental_sustainability': False, 'economic_sustainability': False}


In [22]:
print(result.match)

['indirect']


In [24]:
SeeSus.show_syntax("SDG2_1")

[{'SDG_id': 'SDG2_1', 'SDG_keywords': '(sdg|goal|target|indicator)[^0-9]{0,2}(?=2\\.1[\\.]{0,1})', 'match_type': 'direct'}, {'SDG_id': 'SDG2_1', 'SDG_keywords': '^(?=.*(?:alleviat\\S*|\\babate\\S*|abridge\\S*|\\bavert\\S*|abolish\\S*|eradicat\\S*|eliminat\\S*|\\beras\\S*|\\bease\\S*|\\bend.?\\b|\\bended\\b|rid of|phase out|\\bwipe out|\\breduc\\S*|\\bcut.?\\b|\\bcurb\\S*|curtail\\S*|declin\\S*|decreas\\S*|diminish\\S*|diminution\\S*|\\bdrop\\S*|dwindl\\S*|\\bfade\\b|fall off|go down|goes down|went down|tone down|die down|cut down|\\bhalt\\S*|hinder\\S*|\\bhamper\\S*|impede\\S*|\\binhibit.?\\b|\\binhibited\\b|\\binhibiting|lessen\\S*|mitigat\\S*|slash\\S*|shrink\\S*|\\bstop\\S*|slump\\S*|\\btrim\\S*|weaken\\S*|\\bno\\b|\\bzero|prevent\\S*|prohibit\\S*|minimis\\S*|minimiz\\S*))(?=.*(?:hunger|undernourish\\S*|undernutrion|starv\\S*|famine\\S*|malnourish\\S*|malnutrition\\S*)).+', 'match_type': 'indirect'}, {'SDG_id': 'SDG2_1', 'SDG_keywords': '^(?=.*(?:alleviat\\S*|\\babate\\S*|abridge\\S

In [27]:
SeeSus.edit_syntax("SDG2_1", "my match terms")

The indirect match syntax of SDG2_1 has been updated.


In [28]:
SeeSus.show_syntax("SDG2_1")

[{'SDG_id': 'SDG2_1', 'SDG_keywords': '(sdg|goal|target|indicator)[^0-9]{0,2}(?=2\\.1[\\.]{0,1})', 'match_type': 'direct'}, {'SDG_id': 'SDG2_1', 'SDG_keywords': 'my match terms', 'match_type': 'indirect'}, {'SDG_id': 'SDG2_1', 'SDG_keywords': 'my match terms', 'match_type': 'indirect'}, {'SDG_id': 'SDG2_1', 'SDG_keywords': 'my match terms', 'match_type': 'indirect'}]


In [30]:
new_result = SeeSus("my match terms")

In [32]:
print(new_result.sdg)

['SDG2']

In [33]:
print(new_result.target)

['SDG2_1']
